# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [9]:
# Your code here; import the necessary packages

import pandas as pd
import sqlite3


In [11]:
# Your code here; create the connection
conn = sqlite3.connect("data.sqlite")

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [12]:
# Your code here

q = ("""
 SELECT 
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
WHERE customerNumber IN (
    SELECT customerNumber
    FROM orders
    WHERE orderDate = '2003-01-31'
)  
"""
)

pd.read_sql(q,conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [13]:
# Your code here

q = ("""
SELECT 
    p.productName AS "Product Name",
    COUNT(DISTINCT od.orderNumber) AS "Total Orders",
    SUM(od.quantityOrdered) AS "Total Quantity Sold"
FROM orderdetails od
JOIN products p ON od.productCode = p.productCode
GROUP BY p.productName
ORDER BY "Total Quantity Sold" DESC;
""")
pd.read_sql(q,conn)

,Product Name,Total Orders,Total Quantity Sold
0,1992 Ferrari 360 Spider red,53,1808
1,1937 Lincoln Berline,28,1111
2,American Airlines: MD-11S,28,1085
3,1941 Chevrolet Special Deluxe Cabriolet,28,1076
4,1930 Buick Marquette Phaeton,28,1074
...,...,...,...
104,1999 Indy 500 Monte Carlo SS,25,855
105,1911 Ford Town Car,25,832
106,1936 Mercedes Benz 500k Roadster,25,824
107,1970 Chevy Chevelle SS 454,25,803


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [20]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
"""
products --> orderdetails  ---> orders ---> customers

Select the Product Name and the  Total Number of People Who Have Ordered Each Product
Sort the results in descending order.
"""

q = ("""
SELECT 
    products.productCode AS "Product Code",
    products.productName AS "Product Name",
    COUNT(DISTINCT customers.customerNumber) AS "Customers Purchased"
FROM products
JOIN orderdetails
    ON products.productCode = orderdetails.productCode
JOIN orders
    ON orderdetails.orderNumber = orders.orderNumber
JOIN customers
    ON orders.customerNumber = customers.customerNumber
GROUP BY products.productName
ORDER BY "Customers Purchased" DESC;
""") 

pd.read_sql(q,conn)

,Product Code,Product Name,Customers Purchased
0,S18_3232,1992 Ferrari 360 Spider red,40
1,S72_1253,Boeing X-32A JSF,27
2,S10_4757,1972 Alfa Romeo GTA,27
3,S10_1949,1952 Alpine Renault 1300,27
4,S18_2957,1934 Ford V8 Coupe,27
...,...,...,...
104,S24_2840,1958 Chevy Corvette Limited Edition,19
105,S24_3432,2002 Chevy Corvette,18
106,S24_3191,1969 Chevrolet Camaro Z28,18
107,S24_2887,1952 Citroen-15CV,18


In [22]:
q_orderdetails = ("""
SELECT * 
FROM orderdetails
""")

q_orders = ("""
SELECT * 
FROM orders
""")

q_products = ("""
SELECT * 
FROM products
""")

q_customers = ("""
SELECT * 
FROM customers             
""")
print(pd.read_sql(q_products, conn).columns)
print(pd.read_sql(q_orderdetails, conn).columns)
print(pd.read_sql(q_orders, conn).columns)
print(pd.read_sql(q_customers, conn).columns)

Index(['productCode', 'productName', 'productLine', 'productScale',
       'productVendor', 'productDescription', 'quantityInStock', 'buyPrice',
       'MSRP'],
      dtype='object')
Index(['orderNumber', 'productCode', 'quantityOrdered', 'priceEach',
       'orderLineNumber'],
      dtype='object')
Index(['orderNumber', 'orderDate', 'requiredDate', 'shippedDate', 'status',
       'comments', 'customerNumber'],
      dtype='object')
Index(['customerNumber', 'customerName', 'contactLastName', 'contactFirstName',
       'phone', 'addressLine1', 'addressLine2', 'city', 'state', 'postalCode',
       'country', 'salesRepEmployeeNumber', 'creditLimit'],
      dtype='object')


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [ ]:
# Your code here
q = ("""
SELECT DISTINCT
    employees.employeeNumber,
    employees.firstName,
    employees.lastName,
    offices.city,
    offices.officeCode
FROM employees
JOIN customers
    ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders
    ON customers.customerNumber = orders.customerNumber
JOIN orderdetails
    ON orders.orderNumber = orderdetails.orderNumber
JOIN products
    ON orderdetails.productCode = products.productCode
JOIN offices
    ON employees.officeCode = offices.officeCode
WHERE products.productCode IN (
    SELECT 
        products.productCode
    FROM products
    JOIN orderdetails ON products.productCode = orderdetails.productCode
    JOIN orders ON orderdetails.orderNumber = orders.orderNumber
    JOIN customers ON orders.customerNumber = customers.customerNumber
    GROUP BY products.productCode
    HAVING COUNT(DISTINCT customers.customerNumber) < 20
)

""")

pd.read_sql(q, conn)

## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [36]:
# Your code here

q = ("""
SELECT DISTINCT
    employees.employeeNumber,
    employees.firstName,
    employees.lastName,
    COUNT(DISTINCT customers.customerNumber)
FROM employees
JOIN customers
    ON employees.employeeNumber = customers.salesRepEmployeeNumber
WHERE employees.employeeNumber IN (
    SELECT salesRepEmployeeNumber
    FROM customers
    GROUP BY salesRepEmployeeNumber
    HAVING AVG(creditLimit) > 15000
)
GROUP BY employees.employeeNumber
""")
pd.read_sql(q, conn)


,employeeNumber,firstName,lastName,COUNT(DISTINCT customers.customerNumber)
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7
5,1323,George,Vanauf,8
6,1337,Loui,Bondur,6
7,1370,Gerard,Hernandez,7
8,1401,Pamela,Castillo,10
9,1501,Larry,Bott,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!